In [1]:
from datetime import *
import urllib.request as req
import requests
import bs4
import time
import pandas as pd
import numpy as np
import re
import string

In [2]:
### 變數及參數設定 ###

# 網址
url_1 = "https://news.tvbs.com.tw/news/searchresult/%E8%A9%90%E4%BF%9D/news" # 詐保
url_2 = "https://news.tvbs.com.tw/news/searchresult/%E8%B2%AA%E6%B1%99/news" # 貪汙
url_3 = "https://news.tvbs.com.tw/news/searchresult/%E5%85%AC%E5%8B%99%E5%93%A1%20%E8%B2%AA%E6%B1%99/news" # 公務員 貪汙
url_4 = "https://news.tvbs.com.tw/news/searchresult/%E5%85%A7%E7%B7%9A%E4%BA%A4%E6%98%93/news" # 內線交易
url_5 = "https://news.tvbs.com.tw/news/searchresult/%E6%AC%A0%E7%A8%85%E5%A4%A7%E6%88%B6/news"  # 欠稅大戶

# 幾日前
days = 7

# 一頁多少篇新聞
one_page = 25

# 獲得現在日期時刻
today = datetime.today()
print(today)

# 某日日期
## today = datetime.strptime("2024-05-31 23:59:59", "%Y-%m-%d %H:%M:%S")

2025-05-15 15:09:54.129304


In [3]:
def TVBS(url, days, today, one_page):
    ## 建立 list ##
    Key = []        # 關鍵字
    Title = []      # 標題
    Time = []       # 發表日期
    Text = []       # 內文
    URL = []        # 網址

    response = requests.get(url)
    original_page = bs4.BeautifulSoup(response.text, "html.parser")
    news_num_label = original_page.find("li", class_ = "t2").find("div", class_ = "word").find_all("span")
    key_name = news_num_label[0].string
    news_num = int(re.findall(r"\d+", news_num_label[1].string)[0])
    pages = news_num//one_page+1

    for page in range(1, pages+1):
        main_page = requests.get(url + "/" + str(page)).text
    
        main_page = bs4.BeautifulSoup(main_page, "html.parser")

        title_label = main_page.find_all("h2", class_ = "txt")
        time_label = original_page.find("div", class_ = "news_list").find("div", class_ = "list").ul.find_all("li")
        for i in range(one_page):
            title = title_label[i].string
            time = time_label[i].find("div", class_ = "time").string

            if "前" in time:
                news_url = time_label[i].a["href"]
                Key.append(key_name), Title.append(title), URL.append(news_url)
                news_response = requests.get(news_url)
                news_page = bs4.BeautifulSoup(news_response.text, "html.parser")
                news_time_string = news_page.find("div", class_ = "author_box").find("div", class_ = "author").text
                # news_time_string = news_time_string.replace("\t", "").replace("\n", "")
                news_time = re.findall(r"\d{4}/\d{2}/\d{2} \d{2}:\d{2}", news_time_string)[0]
                news_time = datetime.strptime(news_time, "%Y/%m/%d %H:%M")
                Time.append(news_time)
                news = news_page.find("div", class_ = "article_content").text.replace("\n", "").replace("\t", "")
                Text.append(news)
        
            else:
                time = datetime.strptime(time, "%Y/%m/%d %H:%M")
                news_url = time_label[i].a["href"]
    
                sec = (today - time).total_seconds()
                days_sec = timedelta(days = days).total_seconds()
    
                if sec <= days_sec and sec >= 0:
                        Key.append(key_name), Title.append(title), Time.append(time), URL.append(news_url)
    
                        news_response = requests.get(news_url)
                        news_page = bs4.BeautifulSoup(news_response.text, "html.parser")
                        news = news_page.find("div", class_ = "article_content").text.replace("\n", "").replace("\t", "").replace("\xa0", "")
                        Text.append(news)
                
                elif sec < 0:
                    continue
                    
                else:
                    break
                    
        else:
            continue
            
        break
        
    # 輸出 DataFrame
    data = pd.DataFrame({
        "關鍵字":Key,
        "標題":Title,
        "時間":Time,
        "內文":Text,
        "網址":URL
    })
    
    return data

In [5]:
TVBS(url_2, days, today, one_page)

,關鍵字,標題,時間,內文,網址
0,貪汙,大陸半導體狂人 紫光前董座趙偉國貪汙判死緩 貪汙逾56.8億台幣,2025-05-15 14:00:00,被大陸稱為「半導體狂人」的中國紫光集團前董事長趙偉國，因為貪汙超過56.8億台幣，被判處死刑...,https://news.tvbs.com.tw/china/2870964
1,貪汙,放寬中藥商資格引藥師怒吼 控衛福部長、司長涉貪汙、圖利,2025-05-12 18:36:00,衛福部3月18日發布解釋令，放寬中藥從業人員資格，引發藥師公會全聯會強烈不滿，認為此舉違反《...,https://news.tvbs.com.tw/local/2867919


In [6]:
TVBS(url_3, days, today, one_page)

,關鍵字,標題,時間,內文,網址


In [8]:
TVBS(url_4, days, today, one_page)

,關鍵字,標題,時間,內文,網址


In [9]:
TVBS(url_5, days, today, one_page)

,關鍵字,標題,時間,內文,網址
